In [1]:
!pip install nltk
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install wikipedia

import nltk
import numpy as np
import random
import string
import wikipedia
import re


     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 8.2MB 6.0MB/s 
     |████████████████████████████████| 7.1MB 5.6MB/s 
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11686 sha256=cf695639449763bbdf9471c2b5606b16e274df95e879502087bf03d489af7b7a
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [31]:
wikipedia.set_lang("ru")
p = wikipedia.page("Космос")
content = p.content # Content of page.

nltk.download('punkt')
nltk.download('stopwords')

#Это почистит заголовки
result = re.findall(".*?(==)(.*?)(==)", content)
for s in result:
  content = content.replace("".join(s),"")

sent_tokens = nltk.sent_tokenize(content)
word_tokens = nltk.word_tokenize(content)

sent_tokens[:10]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['Косми́ческое простра́нство, ко́смос (др.-греч.',
 'κόσμος — «мир», «Вселенная») — относительно пустые участки Вселенной, которые лежат вне границ атмосфер небесных тел.',
 'Космос не является абсолютно пустым пространством: в нём есть, хотя и с очень низкой плотностью, межзвёздное вещество (преимущественно молекулы водорода), кислород в малых количествах (остаток после взрыва звезды), космические лучи и электромагнитное излучение, а также гипотетическая тёмная материя.',
 'В своём изначальном понимании греческий термин «космос» (порядок, мироустройство) имел философскую основу, определяя гипотетический замкнутый вакуум вокруг Земли — центра Вселенной.',
 'Тем не менее, в языках на латинской основе и её заимствованиях к одинаковой семантике применяют практический термин «пространство» (так как с научной точки зрения обволакивающий Землю вакуум бесконечен), поэтому в русском и близких ему языках в результате реформенной корректировки родился своеобразный плеоназм «космическое пространс

In [40]:
import pymorphy2

lemmer = pymorphy2.MorphAnalyzer()

In [41]:
def LemTokens(tokens):
    return [lemmer.parse(token)[0].normal_form for token in tokens]
    # return [eng_lemmer.lemmatize(token) for token in tokens]
    
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [45]:

# Тут нужно менять!
GREETING_INPUTS = ("сап", "салют")
GREETING_RESPONSES = ["Угу", "Ага"]

def greeting(sentence):
    sentence = re.sub('[^А-Яа-яё ]', '', sentence)
    #sentence = re.sub('[^A-Za-z ]', '', sentence)
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [43]:
def response(user_response):
    robo_response=''
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words("russian"))
    # TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english")
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"непонял" # ТУТ можно менять :) 
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [47]:
import warnings
warnings.filterwarnings('ignore')

flag=True

# Тут тоже нужно менять!
print("Робот: Дам битую инфу про космос. Как надоест, напиши \"конец\"")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='все'):
        if(user_response in ['кончай', 'отвали']): # И тут меняйте :)
            flag=False
            print("Робот: славно") # Меняйте :) 
        else:
            if(greeting(user_response)!=None):
                print("Робот: "+greeting(user_response))
            else:
                sent_tokens.append(user_response)
                word_tokens=word_tokens+nltk.word_tokenize(user_response)
                final_words=list(set(word_tokens))
                print("Робот: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Робот: Вали") # Тут тоже нужно импровизировать

Робот: Дам битую инфу про космос. Как надоест, напиши "конец"
Сап
Робот: Угу
Космическое пространство
Робот: Пространство в Солнечной системе называют межпланетным пространством, которое переходит в межзвёздное пространство в точках гелиопаузы солнцестояния.
Смерть в космосе
Робот: Смерть может наступить примерно через 1-2 минуты, хотя точно это не известно.
все
Робот: Вали


In [ ]:
Rasa

sklearn
tensorflow
torch
transformers
mystem, pymorphy2, spacy
gensim